In [ ]:
%pip install -q ipywidgets dotenv tqdm humanize huggingface_hub pandas

In [2]:
import os
import time
import datetime
import tarfile

from pathlib import Path
from csv import QUOTE_NONE
from html import escape

import humanize
import pandas as pd

from huggingface_hub import hf_hub_download

from IPython.display import HTML
from dotenv import load_dotenv

os.environ.setdefault("HF_HUB_DISABLE_HF_TRANSFER", "1")
os.environ.setdefault("HF_HUB_DISABLE_XET", "1")

REPO_ID = "google/fleurs"
LANG_ID = "cs_cz"

LOCAL_DATA_ROOT = Path("data")
LOCAL_DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [3]:
os.environ.setdefault("HF_HUB_DISABLE_HF_TRANSFER", "1")
os.environ.setdefault("HF_HUB_DISABLE_XET", "1")

REPO_ID = "google/fleurs"
LANG_ID = "cs_cz"

LOCAL_DATA_ROOT = Path("data")
LOCAL_DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [4]:
text_tsv_path = Path(
    hf_hub_download(
        repo_id=REPO_ID,
        repo_type="dataset",
        filename=f"data/{LANG_ID}/test.tsv",
        local_dir=LOCAL_DATA_ROOT,
    )
).resolve()

In [5]:
audio_tar_path = Path(
    hf_hub_download(
        repo_id=REPO_ID,
        repo_type="dataset",
        filename=f"data/{LANG_ID}/audio/test.tar.gz",
        local_dir=LOCAL_DATA_ROOT,
    )
).resolve()

In [6]:
audio_root = audio_tar_path.parent
audio_root.mkdir(parents=True, exist_ok=True)

if not any(audio_root.rglob("*.wav")):
    with tarfile.open(audio_tar_path) as tar:
        tar.extractall(path=audio_root, filter="data")

In [7]:
column_names = [
    "sentence_id",
    "path",
    "raw_transcription",
    "normalized_transcription",
    "transcription_tokens",
    "duration_ms",
    "speaker_gender",
]

fleurs_test = pd.read_csv(
    text_tsv_path,
    sep="\t",
    names=column_names,
    header=None,
    quoting=QUOTE_NONE,
    engine="python")

fleurs_test["audio_path"] = fleurs_test["path"].map(lambda relative_path: str(audio_root / "test" / relative_path))
fleurs_test

,sentence_id,path,raw_transcription,normalized_transcription,transcription_tokens,duration_ms,speaker_gender,audio_path
0,1817,5113552872926594975.wav,Schengenský prostor nicméně v tomto ohledu fun...,schengenský prostor nicméně v tomto ohledu fun...,s c h e n g e n s k ý | p r o s t o r | n i c ...,86400,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
1,1817,1644157818502361154.wav,Schengenský prostor nicméně v tomto ohledu fun...,schengenský prostor nicméně v tomto ohledu fun...,s c h e n g e n s k ý | p r o s t o r | n i c ...,112320,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
2,1991,1942487314732498280.wav,Během tohoto období evropské historie se katol...,během tohoto období evropské historie se katol...,b ě h e m | t o h o t o | o b d o b í | e v r ...,126720,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
3,1991,431300690903234852.wav,Během tohoto období evropské historie se katol...,během tohoto období evropské historie se katol...,b ě h e m | t o h o t o | o b d o b í | e v r ...,223680,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
4,1905,13920301186810235641.wav,Hlavním městem Moldávie je Kišiněv. Místním ja...,hlavním městem moldávie je kišiněv místním jaz...,h l a v n í m | m ě s t e m | m o l d á v i e ...,208320,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
...,...,...,...,...,...,...,...,...
718,1704,7128341050772567860.wav,V roce 1990 bylo kvůli pouštnímu písku přidáno...,v roce 1990 bylo kvůli pouštnímu písku přidáno...,v | r o c e | 1 9 9 0 | b y l o | k v ů l i | ...,197760,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
719,1704,6480320536280994455.wav,V roce 1990 bylo kvůli pouštnímu písku přidáno...,v roce 1990 bylo kvůli pouštnímu písku přidáno...,v | r o c e | 1 9 9 0 | b y l o | k v ů l i | ...,154560,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
720,1704,17346462614465166923.wav,V roce 1990 bylo kvůli pouštnímu písku přidáno...,v roce 1990 bylo kvůli pouštnímu písku přidáno...,v | r o c e | 1 9 9 0 | b y l o | k v ů l i | ...,216960,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...
721,1674,636931273120013697.wav,"V některých oblastech stačí vařit vodu minutu,...",v některých oblastech stačí vařit vodu minutu ...,v | n ě k t e r ý c h | o b l a s t e c h | s ...,113280,MALE,/Users/karmi/Contracts/PostBellum/Code/Transcr...


In [8]:
rows: list[str] = []
for transcription, audio_path in fleurs_test[["normalized_transcription", "audio_path"]].head(5).itertuples(index=False):
    text_html = escape(str(transcription).strip())
    audio_uri = audio_path
    rows.append(
        (
            '<li class="sample-item">'
            f'  <p class="transcription">{text_html}</p>'
            f'  <audio controls preload="none" src="{audio_uri}"></audio>'
            '</li>'
        )
    )

style = (
    "<style>"
    ".sample-list {list-style:none;padding:0;margin:0;display:flex;flex-direction:column;gap:1.25rem;"
    'font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",sans-serif;}'
    ".sample-item {display:flex;flex-direction:column;gap:0.4rem;}"
    ".sample-item .transcription {margin:0;line-height:1.55;font-size:1rem;font-weight:500;letter-spacing:0.01em;}"
    ".sample-item audio {width:100%;}"
    "</style>"
)

audio_duration_ms = fleurs_test["duration_ms"].sum()
total_duration_td = pd.to_timedelta(audio_duration_ms, unit="ms").to_pytimedelta()

print(f"Total duration: {humanize.precisedelta(total_duration_td, minimum_unit='hours', suppress=['days'])}")

HTML(f"{''.join(style)}<ol class='sample-list'>" + "".join(rows) + "</ol>")

Total duration: 39.20 hours
